In [1]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/config

env: JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
env: FLINK_CONF_DIR=/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/config


In [2]:
%load_ext streaming_jupyter_integrations.magics

Could not find scli config file at '/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/.streaming_config.yml'. Will not load any secrets.
Set env variable JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'


In [3]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081


remote environment has been created.


In [4]:
%%flink_execute_sql
CREATE TABLE practitionerRole (
  `id` STRING,
  `practitioner` ROW (
    `identifier` ROW (
      `value` STRING
    ),
    `display` STRING
  ),
  `organization` ROW (
    `identifier` ROW (
      `value` STRING
    )
  ),
  `specialty` ROW (
    `text` STRING
  ) ARRAY
) WITH (
  'connector' = 'kafka',
  'topic' = 'PractitionerRole',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

Job starting...
Job started
Execution successful


In [5]:
%%flink_execute_sql
CREATE TABLE practitioner (
  `identifier` ROW (
    `value` STRING
  ) ARRAY,
  `active` BOOLEAN,
  `name` ROW (
    `family` STRING,
    `given` STRING ARRAY
  ) ARRAY,
  `telecom` ROW (
    `value` STRING
  ) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ) ARRAY,
  `gender` STRING
) WITH (
  'connector' = 'kafka',
  'topic' = 'Practitioner',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

Job starting...
Job started
Execution successful


In [6]:
%%flink_execute_sql
CREATE TABLE practitioner_psql (
  `identifier` STRING,
  `name` STRING,
  `email` STRING,
  `adress` STRING,
  `city` STRING,
  `state` STRING,
  `postalCode` STRING,
  `country` STRING,
  `gender` STRING,
  `role` STRING,
  `id_organization` STRING,
  PRIMARY KEY (`identifier`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://db:5432/example',
  'table-name' = 'public.practitioner',
  'username' = 'example',
  'password' = 'example'
);


Job starting...
Job started
Execution successful


In [7]:
%%flink_execute_sql
INSERT INTO practitioner_psql
SELECT
p.identifier[1][1] as identifier,
pr.practitioner.display as name,
telecom[1][1] as email,
p.address[1].line[1] AS adress,  
p.address[1].city AS city, 
p.address[1].state AS state, 
p.address[1].country AS country, 
p.address[1].postalCode AS postalcode, 
p.gender,
pr.specialty[1].text as role,
pr.organization.identifier[1] as id_organization
FROM practitioner p 
JOIN practitionerRole pr
ON p.identifier[1][1] = pr.practitioner.identifier[1]

This job runs in a background, please either wait or interrupt its execution before continuing


Job starting...
Job started
Exception while waiting for rows. An error occurred while calling o166.await.
: java.util.concurrent.ExecutionException: org.apache.flink.table.api.TableException: Failed to wait job finish
	at java.base/java.util.concurrent.CompletableFuture.reportGet(CompletableFuture.java:395)
	at java.base/java.util.concurrent.CompletableFuture.get(CompletableFuture.java:2092)
	at org.apache.flink.table.api.internal.TableResultImpl.awaitInternal(TableResultImpl.java:116)
	at org.apache.flink.table.api.internal.TableResultImpl.await(TableResultImpl.java:90)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:78)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:567)
	at org.apache.flink.api.python.shaded.py4j.reflection.MethodInvok